# 第4讲 描述性统计分析基础
- 数据集描述与属性说明
- ID	客户编号
- Suc_flag	成功入网标识
- ARPU	入网后ARPU
- PromCnt12	12个月内的营销次数
- PromCnt36	36个月内的营销次数
- PromCntMsg12	12个月内发短信的次数
- PromCntMsg36	36个月内发短信的次数
- Class	客户重要性等级(根据前运营商消费情况)
- Age	年龄
- Gender	性别
- HomeOwner	是否拥有住房
- AvgARPU	当地平均ARPU
- AvgHomeValue	当地房屋均价
- AvgIncome	当地人均收入

In [ ]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
#os.chdir('Q:/data')
#os.getcwd()

In [ ]:
# 读取数据
camp= pd.read_csv('teleco_camp.csv')

In [ ]:
# 数据预处理
camp.dtypes

In [ ]:
camp.describe(include='all')

In [ ]:
camp['Suc_flag'] = camp['Suc_flag'].astype('category')
camp['Class'] = camp['Class'].astype('category')

## 3.1 描述性统计与探索型数据分析
- 1.分类变量分析

In [ ]:
camp['Suc_flag'].groupby(camp['Suc_flag']).count()

- 2.连续变量分析
- 2.1.数据的集中趋势
ARPU的均值与中位数


In [ ]:
fs = camp['ARPU'] # 可以使用camp.ARPU 
type(fs)

In [ ]:
print('mean = %6.4f' %fs.mean())                     # 求fs的均值
print('median = %6.4f' %fs.median() )                # 求fs的中位数
print('quantiles\n', fs.quantile([0.25, 0.5, 0.75])) # 求a的上下四分位数与中位数

In [ ]:
%matplotlib inline

fs.plot(kind='hist')

- 2.2.数据的离散程度

In [ ]:
print ('mad = %6.4f' %fs.mad())      # 求平均绝对偏差 mad = np.abs(fs - fs.mean()).mean()
print ('range = %6.4f' %(fs.max(skipna=True) - fs.min(skipna=True))) # 求极差
print ('var = %6.4f' %fs.var())   # 求方差
print ('std = %6.4f' %fs.std())   # 求标准差

- 2.3.数据的偏度与峰度-
ARPU:右偏

In [ ]:
import matplotlib.pyplot as plt

plt.hist(fs.dropna(), bins=15)

In [ ]:
print ('skewness = %6.4f' %fs.skew(skipna=True))
print ('kurtosis = %6.4f' %fs.kurt(skipna=True))

## 3.2 apply\map\groupby及其它相关

In [ ]:
data = pd.DataFrame(data={'a':range(1,11), 'b':np.random.randn(10)})
data.T

In [ ]:
data.apply(np.mean) # 等价于data.mean()，是其完整形式

In [ ]:
data.apply(lambda x: x.astype('str')).dtypes # DataFrame没有astype方法，只有Series有

In [ ]:
(data['a']- data['a'].mean()) / data['a'].std()

In [ ]:
data['a'].map(lambda x: (x - data['a'].mean()) / data['a'].std())  # 等价于(data['a']- data['a'].mean()) / data['a'].std()

In [ ]:
data['a'].map(lambda x: int(str(x), base=16))  # 不支持“广播”时，可以使用map进行函数映射

分组-应用/聚合

In [ ]:
key = [1, 2] * 5
key

In [ ]:
group1, group2 = data.groupby(key) # 使用groupby可按照‘key’进行分组，‘key’需与待分组数据有同样长度
print (group1)
print (group2)

In [ ]:
data.groupby(key).aggregate(np.mean) 
# 聚合函数在各分组中进行聚合，是data.groupby(key).mean()的完整形式，可传入函数或字符串(sum/mean/median/std/var等)，也可传入列表

In [ ]:
data.groupby(key).agg({'a': 'sum', 'b':'count'}) # agg可以在多列上使用不同的聚合函数

In [ ]:
data.groupby(key).transform(np.mean) # 转换函数可在各分组内进行运算，将结果广播到原数据中

In [ ]:
data.groupby(key).apply(np.mean) # apply是一般化的‘分组-应用/聚合’函数，更灵活地实现aggregate或transform的功能

*练习：对camp数据集，按照客户级别汇总ARPU

In [ ]:
camp['ARPU'].groupby(camp['Class']).apply(lambda x: x.describe())

In [ ]:
camp[['PromCnt12','PromCnt36']].groupby(camp['Suc_flag']).mean()

* crosstab 和pivot_table

In [ ]:
pd.crosstab(camp.Suc_flag, camp.Class)

In [ ]:
camp.pivot_table(['PromCnt12','PromCnt36'], 
                    index='Suc_flag', columns='Class', aggfunc=np.mean) # index、columns、aggfunc参数均可传入列表

## 3.3绘图
- pandas.plot是matplotlib.pyplot.plot的简单包装

散点图

In [ ]:
camp.plot(x='AvgARPU', y='ARPU', kind='scatter') #散点图
plt.text(60, 1100, 'ARPU VS AvgARPU') # 加标签

柱图/条形图、折线图

In [ ]:
data.b.plot(kind='bar') # 柱图
plt.show()

In [ ]:
data.b.plot(kind='line') # 折线图
plt.show()

直方图

In [ ]:
camp[['PromCnt12','PromCnt36']].plot(stacked=False, kind='hist', alpha=0.3, bins=20)

In [ ]:
camp[['PromCnt12','PromCnt36']].hist(alpha=0.8, bins=20, figsize=(10, 2))

 饼图

In [ ]:
gb = camp['Suc_flag'].groupby(camp['Suc_flag']).count()
gb.plot(kind='pie',labels=['Yes', 'No'], colors=['r', 'g'], autopct='%.2f', fontsize=20, figsize=(6, 6))

箱线图

In [ ]:
camp[['PromCnt12','Suc_flag']].boxplot(by='Suc_flag')